# Titanic Challenge

Resolveremos el ejercicio que se puede encontrar en:

https://www.kaggle.com/c/titanic-gettingStarted

A continuacion agregare la descripcion de las variables. Esta informacion se puede encontrar en la pagina previamente mencionada. Por conveniencia se traduciran todas las etiquetas y variables a idioma espanol.

DESCRIPCION DE LAS VARIABLES:
    
    Sobrevivencia   Sobrevivencia (0 = No; 1 = Si) 
    Clase           Clase del Pasajero
                    (1 = 1ra; 2 = 2da; 3 = 3ra)
    Nombre          Nombre
    Sexo            Sexo
    Edad            Edad
    Sibsp           Numero de Hijos/Esposos Abordo
    Parch           Numero de Padres/Ninos Abordo
    Boleto          Numero del Boleto
    Tarifa          Tarifa del Pasajero
    Cabina          Cabina
    Embarco         Puerto Donde Emabarco
                    (C = Cherbourg; Q = Queenstown; S = Southampton)

NOTAS ESPECIALES:

    Clase es un aproximado de su estado socio economico (SES)
     1st ~ Alta; 2nd ~ Media; 3rd ~ Baja

    La Edad esta en anios; Es fraccional si la edad es menor a uno (1)
     Si la edad es estimada, estara en la forma xx.5

In [2]:
#Importamos todas las librerias que necesitamos para analisis y visualizacion
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas import Series, DataFrame

%matplotlib inline

In [3]:
#Cargamos los datos a un DataFrame
titanic_df = pd.read_csv('train_titanic.csv')

In [4]:
#Demos una vista a nuestros datos
titanic_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [ ]:
#Obtendremos informacion general de nuestros datos
titanic_df.info()

##### Podemos observar que en la columna Age y Cabin existen valores nulos, debido a la falta significativa de datos en Cabin, solo limpiaremos la columna Age

In [ ]:
#Sustituimos los valores nulos con la mediana
index = titanic_df['Age'].isnull()
edadMedia = titanic_df['Age'].median()
titanic_df.loc[index, 'Age'] = edadMedia

In [ ]:
#Checamos la transformacion de nuestros dat-os
titanic_df.head(10)

## Para entender mejor el problema, resolvamos las siguientes preguntas:
    1.) Quienes eran los pasajeros del Titanic? (Edad,Genero,Clase,..etc)
    2.) En que cubierta estaban los pasajeros?
    3.) De donde vienen los pasajeros?
    4.) Quien estaba solo y quienes tenian familia?
    5.) Que factores ayudaron a sobrevivir del hundimiento?

### 1.) Quienes eran los pasajeros del Titanic? (Edad,Genero,Clase,..etc)

In [ ]:
#Chequemos el genero
g = sns.factorplot('Sex',data=titanic_df, kind='count', palette='Dark2', size=5, aspect=.8)
(g.set_axis_labels("", "Cantidad")
.set_xticklabels(["Hombre","Mujer"]))

In [ ]:
#Separemos el genero por clases
g = sns.factorplot('Pclass', data=titanic_df,hue='Sex',kind='count', palette='Dark2', size=5)
(g.set_axis_labels("Clase", "Cantidad")
.set_xticklabels(["Primera","Segunda","Tercera"]))

Un descubrimiento interesante que podemos observar es que hay muchos mas hombres en 3ra clase que mujeres, en relacion a las demas clases. Sin embargo seria util separarlos entre mujeres, hombres y niños.

In [ ]:
#Creamos una funcion para determinar si son niños o no (Asumiremos que menores de 16 años son niños)
def type_passenger(passenger):
    age,sex = passenger
    if age < 16:
        return 'child'
    else:
        return sex

In [ ]:
#Aplicamos nuestra funcion a una nueva columna
titanic_df['TypeP'] = titanic_df[['Age','Sex']].apply(type_passenger,axis=1)

In [ ]:
#Demos una vista a nustros datos
titanic_df.head(10)

Perfecto! Hemos separado a los pasajeros entre niños, mujeres y hombres.

In [ ]:
#Ahora grafiquemos agrupandolo por tipo de persona
g = sns.factorplot('Pclass', data=titanic_df, hue='TypeP', kind='count', palette='Dark2', size=5)
(g.set_axis_labels("Clase", "Cantidad")
.set_xticklabels(["Primera","Segunda","Tercera"]))

Se puede observar que hay bantantes niños en 3ra clase y muy pocos en 1ra. 

A continuacion crearemos distribuciones de edades para conocer mas informacion acerca de los pasajeros.

In [ ]:
sns.distplot(titanic_df['Age'], kde=False, color='g', bins=8)

In [ ]:
#Obtenemos una comparasion general entre niños, mujeres y hombres
titanic_df['TypeP'].value_counts()

In [ ]:
#Otra manera de visualizar los datos es graficando los KDE (Calculando la Funcion de Densidad de Probablidad) 
#en una misma grafica
fig = sns.FacetGrid(titanic_df, hue='Sex', aspect=4, palette='Dark2')
fig.map(sns.kdeplot,'Age', shade=True)
oldest = titanic_df['Age'].max()
fig.set(xlim=(0,oldest))
fig.add_legend()

In [ ]:
#Ahora usamos Tipo de Persona
fig = sns.FacetGrid(titanic_df, hue='TypeP', aspect=4, palette='Dark2')
fig.map(sns.kdeplot,'Age', shade=True)
oldest = titanic_df['Age'].max()
fig.set(xlim=(0,oldest))
fig.add_legend()

In [ ]:
#Ahora por clase
fig = sns.FacetGrid(titanic_df, hue='Pclass', aspect=4, palette='Dark2')
fig.map(sns.kdeplot,'Age', shade=True)
oldest = titanic_df['Age'].max()
fig.set(xlim=(0,oldest))
fig.add_legend()

Obtuvimos bastante informacion acerca de como eran los pasajeros basados en su genero, edad y clase.

### 2.) En que cubierta estaban los pasajeros?

In [ ]:
#Volvemos a observar nuestros datos
titanic_df.head()

Solo necesitamos la primera letra de la cubierta para clasificar su nivel (ej. A,B,C,D,E,F,G)

In [ ]:
#Creamos una nueva columna con el formato deseado
titanic_df['Deck'] = titanic_df['Cabin'][titanic_df['Cabin'].notnull()].astype(str).map(lambda x: x[0])

In [ ]:
titanic_df.head()

In [ ]:
sns.factorplot('Deck', data=titanic_df, palette='bone', kind='count', size=5)

In [ ]:
#Eliminamos los datos de la cubierta "T" que no hacen sentido a nuestro conjunto de datos
titanic_df['Deck'].loc[titanic_df['Deck'] == 'T'] = np.NaN
sns.factorplot('Deck', data=titanic_df, palette='bone', kind='count', order=['A','B','C','D','E','F','G'], size=5)

Muy bien, ahora conocemos la distribucion de los pasajeros. Hay que recordar que solo tenemos 204 valores de los 890 pasajeros.

### 3.) De donde vienen los pasajeros?

In [ ]:
titanic_df.head()

Nota: La columna Embarcado tiene tres valores C,Q y S. Estos hacen referencia a Cherbourg, Queenstown, Southhampton.

In [ ]:
sns.factorplot('Embarked', data=titanic_df, hue='Pclass',order=['C','Q','S'], kind='count', palette='Dark2', size=5)

Un hallazgo interesante es que en Queenstown, casi todo los pasajeros que abordaron son de tercera clase. Podria ser interesante investigar la razon de este suceso, quiza la economia de ese lugar en ese periodo de tiempo.

Tambien podemos visualizar los datos de la siguiente forma para conocer la distribucion de sus edades respecto a su clase y Embarque

In [ ]:
sns.factorplot(x='Age', y='Embarked', hue='Sex', row='Pclass',data=titanic_df[titanic_df.Embarked.notnull()],
                orient="h", size=2, aspect=3.5, palette="Dark2",kind="violin", split=True, cut=0, bw=.2)

### 4.) Quien estaba solo y quienes tenian familia?

Para determinar si una persona esta sola o no, sumaremos los campos de los familiares.

In [ ]:
#Agregamos una nueva columna para definir quienes estan solos
titanic_df['Alone'] = titanic_df.Parch + titanic_df.SibSp

In [ ]:
#Asignamos valores
titanic_df['Alone'].loc[titanic_df['Alone'] > 0] = 'With Family'
titanic_df['Alone'].loc[titanic_df['Alone'] == 0] = 'Alone'

In [ ]:
titanic_df.head()

In [ ]:
#Ahora grafiquemos nuestros datos
sns.factorplot('Alone',data=titanic_df, kind='count', palette='Dark2',size=5)

Podemos observar que la mayoria de los pasajeros estaban solos

### 5.) Que factores ayudaron a sobrevivir del hundimiento?

Primero hay que determinar cuantos pasajeros fallecieron

In [ ]:
#Creamos una nueva columna con el proposito de que sea legible
titanic_df['Survivor'] = titanic_df.Survived.map({0: 'No', 1: 'Yes'})

#Graficamos el numero de personas que sobrevivieron
sns.factorplot('Survivor',data=titanic_df, palette='Set1', kind='count', size=5)

Podemos observar que hubo mas fallecidos que supervivientes.

Veamos si la clase de los pasajeros tiene un efecto en su tasa de supervivencia.

In [ ]:
#Graficamos considerando la clase
sns.factorplot('Pclass', 'Survived', data=titanic_df, size=5)

La tasa de supervivencia de la 3ra clase es substancialmente baja. Pero posiblemente este efecto es causado por el gran numero de hombres en la 3ra clase. Agregaremos a nuestra grafica la clasificacion de las personas para determinar si eran niños, mujeres u hombres.

In [ ]:
#Graficamos considerando la clase y tipo de persona
sns.factorplot('Pclass', 'Survived',hue='TypeP', data=titanic_df, palette='Dark2', size=5)

De estos datos podemos observar que ser hombre o pertenecer a la 3ra clase no son favorables para la sobrevivencia. Incluso sin tener en cuenta la clase, el hecho de ser hombre disminuye dramaticamente las probabilidades de sobrevivir.

Que hay de la edad? Ser mas joven o viejo tiene un efecto en la tasa de supervivencia?

In [ ]:
#Usemos una grafica lineal para representar la edad contra la sobrevivencia
sns.lmplot('Age', 'Survived', data=titanic_df, palette='Set1', size=5)

La tendencia general es que entre mas viejo fuera el pasajero, es menos probable que sobreviviera.

Agregaremos mas parametros para determinar si la clase tiene un efecto en la tasa de sobrevivencia.

In [ ]:
#Separamos la grafica por clases
sns.lmplot('Age', 'Survived', data=titanic_df, hue='Pclass', palette='Dark2', size=5)

In [ ]:
#Usaremos rangos para obtener una visualizacion mas limpia
generations=[10,20,40,60,80]
sns.lmplot('Age','Survived',hue='Pclass',data=titanic_df,palette='Dark2',x_bins=generations, size=5)

Algo interesante a destacar es que las personas mas viejas de 1ra clase tengan una desviacion estandar tan alta.

A continuacion relacionaremos el genero y la edad.

In [ ]:
#Ahora separamos la grafica por sexos
sns.lmplot('Age', 'Survived', hue='Sex', data=titanic_df, palette='Dark2', x_bins=generations, size=5)

La tendencia nos muestra que ser mujer de mayor edad aumentaba las probabilidades de sobrevivir, por el contrario ser hombre dismuniye las probabilidades de sobrevivir, sin embargo como vimos en un caso anterior, las personas de mayor edad tienen una desviacion estandar alta.

Podriamos preguntarnos si las cubiertas tienen algun efecto en la tasa de supervivencia?

In [ ]:
#Graficamos el numero de supervivientes por Cubierta
sns.factorplot('Deck',data=titanic_df,hue='Survivor',kind='count',
               order=['A','B','C','D','E','F','G'],palette='Dark2', hue_order=['Yes','No'])

In [ ]:
sns.factorplot('Deck', 'Survived',data=titanic_df,
               order=['A','B','C','D','E','F','G'],palette='Dark2')

Podriamos pensar que las cubiertas B, D y E tienen la tasa mas alta de supervivencia, pero recordemos que no tenemos todos los datos de los pasajeros. Como mencionamos anteriormente solo tenemos 204 valores de los 890.

Revisemos un poco mas a detalle esos datos, por ejemplo a que clase pertenecen?

In [ ]:
sns.factorplot(y='Deck',data=titanic_df,hue='Pclass',kind='count',
               order=['A','B','C','D','E','F','G'],palette='Dark2')

A simple vista se puede ver que hay mucho mas datos de 1ra clase que de las demas, investiguemos un poco mas.

In [ ]:
#Agrupemos nuestros datos para determinar la cantidad de datos de cabina que tenemos por clase
datos_df = titanic_df.groupby('Pclass')['Deck'].value_counts(normalize=True).reset_index().groupby('Pclass')  \
.sum().reset_index(level=0)
datos_df.columns = ['Pclass', 'Data']
datos_df['Total'] = 1

In [ ]:
#Ahora graficamos
f, ax = plt.subplots()
sns.barplot('Total', 'Pclass',  data=datos_df, color='grey', orient='h', label='Total')
sns.barplot('Data', 'Pclass',  data=datos_df, palette='Set2', orient='h', label='Datos')
ax.set(xlim=(0, 1), ylabel="Clase", xlabel="Datos Obtenidos")

Claramente podemos observar que tenemos un poco mas del 80% de los datos de 1ra clase, mientras que para las demas clases los datos obtenidos no alcanza el 20%, por lo tanto no es posible hacer una conclusion sobre el efecto que tuvieron las cubiertas.

Por ultimo revisemos si estar solo tenia un efecto en la tasa de superviencia.

In [ ]:
#Agrupemos por sexo
sns.factorplot('Alone', 'Survived', hue='TypeP', data=titanic_df, palette='Dark2', size=5)

Observamos que los hombres tenian un ligera mejor posibilidad de sobrevivir si tenian familia, por el contrario las mujeres muestran una tendencia opuesta. El sexo sigue siendo un factor determinante para decidir si sobrevive o no el pasajero.

In [ ]:
#Agrupemos por clase
sns.factorplot('Alone', 'Survived', hue='Pclass', data=titanic_df, palette='Dark2', size=5)

Podemos observar que la clase sigue influyendo significativamente en la tasa de supervivencia y tambien que estar solo disminuye ligeramente las probabilidades de sobrevivir.

En conclusion podemos decir que los factores que mas influyeron significativamente en la supervivencia de los pasajeros, fueron el sexo, la clase y edad, mientras que la cantidad de familiares o donde embarcaron afectan en menor medida.

### Machine Learning

Usemos el algoritmo random forest para hacer predicciones, determinar su precision y caracteriticas mas relevantes

In [ ]:
#Observamos nuestros datos
titanic_df.head()

In [ ]:
#Seleecionamos las caracteristicas que usaremos en nuestro modelo
features = ['Survived','Pclass','TypeP','Age','Embarked','Alone']
train = titanic_df[features]
train.head(10)

In [ ]:
#Checamos que no existan nulos
train.info()

In [ ]:
#Retiramos los valores nulos
train.dropna(inplace=True)

In [ ]:
#Agregamos imports necesarios
import sklearn.ensemble as ske
from patsy import dmatrices

#Creamos una formula para la matriz de diseño de nuestro modelo
#l = [3,2,1]
formula = 'Survived ~ C(Pclass) + C(TypeP) + Age + C(Embarked) + C(Alone)'

In [ ]:
#Creamos las matrices de diseño
y, x = dmatrices(formula, data=train, return_type='dataframe')
y = np.asarray(y).ravel()

#Creamos nuestro random forest
results_rf2 = ske.RandomForestClassifier(n_estimators=100).fit(x,y)

#Obtenemos la puntuacion de presicion y caracteristicas
score = results_rf2.score(x,y)
importances = results_rf2.feature_importances_

In [ ]:
x.head()

In [ ]:
#agrupamos para darle un mejor estilo a la grafica
groupImportances = [importances[1:3].sum(), importances[3:5].sum(), importances[5:7].sum(), importances[7], importances[8]]
d = {'Feature': Series(['Pclass','TypeP', 'Embarked', 'Alone', 'Age']),
    'Values': Series(groupImportances)}
importances_df = DataFrame(d).sort_values(by='Values', ascending=False)

In [ ]:
#Graficamos las caracteristicas mas importantes
sns.barplot('Feature', 'Values',  data=importances_df, palette='Dark2')

In [ ]:
print ("La presicion de las predicciones del Random Forest es: {0}".format(score))

Podemos observar que el comportamiento de nuestro random forest es muy acertado